<a href="https://colab.research.google.com/github/gecon1/dw_matrix_road_sign/blob/master/t3_day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [4]:
cd /content/drive/My Drive/Colab Notebooks/matrix/matrix_tree/dw_matrix_road_sign/

/content/drive/My Drive/Colab Notebooks/matrix/matrix_tree/dw_matrix_road_sign


In [19]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim ==1: y_train = to_categorical(y_train)
if y_test.ndim ==1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_test, y_test)),
      callbacks= [tensorboard_callback]
      
      )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
    ])

In [14]:
model =  get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train,)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 3s 10ms/step - loss: 2.5984 - accuracy: 0.2919 - val_loss: 1.2285 - val_accuracy: 0.6338
Epoch 2/5
272/272 [==============================] - 3s 9ms/step - loss: 0.8099 - accuracy: 0.7445 - val_loss: 0.3918 - val_accuracy: 0.8846
Epoch 3/5
272/272 [==============================] - 3s 9ms/step - loss: 0.3771 - accuracy: 0.8810 - val_loss: 0.1915 - val_accuracy: 0.9438
Epoch 4/5
272/272 [==============================] - 3s 9ms/step - loss: 0.2503 - accuracy: 0.9248 - val_loss: 0.1917 - val_accuracy: 0.9569
Epoch 5/5
272/272 [==============================] - 3s 9ms/step - loss: 0.1862 - accuracy: 0.9454 - val_loss: 0.1440 - val_accuracy: 0.9660


0.9659863945578231

In [16]:
model_trained.evaluate(X_test, y_test)[1]


138/138 [==============================] - 0s 3ms/step - loss: 0.1450 - accuracy: 0.9660


0.9659863710403442

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
    ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=128, #int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0     
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model }

In [31]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
    

}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)


{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4185139871106529, 'dropout_cnn_block_three': 0.3371203305159247, 'dropout_cnn_block_two': 0.4271301106989026, 'dropout_dense_block_one': 0.4342432958219833, 'dropout_dense_block_two': 0.6314069561889963}
accuracy=0.9517006874084473
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3820734014146339, 'dropout_cnn_block_three': 0.4528480175697675, 'dropout_cnn_block_two': 0.4487887565639752, 'dropout_dense_block_one': 0.679333646818541, 'dropout_dense_block_two': 0.3424543816753227}
accuracy=0.9589568972587585
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.496929046763425, 'dropout_cnn_block_three': 0.3585745703188434, 'dropout_cnn_block_two': 0.4143759388654571, 'dropout_dense_block_one': 0.6480977283738217, 'dropout_dense_block_two': 0.4856751999761407}
accuracy=0.8507936596870422
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3161008359173364, 'dropout_cnn_block_three': 0.39597438850519684, 'dropout_cnn_block_two': 0.3120237008481824,